Title

In [2]:
import sys

# 1037 & 335 VS 291 & 51
# Check memory usage
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
sorted([(x, sys.getsizeof(globals().get(x))/1000/1000) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

In [3]:
%timeit
import pandas as pd
import numpy as np

print('loading data...')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')
products = pd.read_csv('../input/products.csv')

orders = pd.read_csv('../input/orders.csv', dtype={'order_id':np.int32,
    'user_id':np.int32,
    'eval_set':'category',
    'order_number':np.int16,
    'order_dow':np.int8,
    'order_hour_of_day':np.int8,
    'days_since_prior_order':np.float32})

train_data = pd.read_csv('../input/order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.int32,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

prior_data = pd.read_csv('../input/order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.int32,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})
print('finished loading')

In [4]:
#orders.groupby('eval_set').user_id.nunique()

In [5]:
print('add order info to priors')
orders.set_index('order_id',inplace=True,drop=False)
prior_data = prior_data.join(orders,on='order_id',rsuffix='_')
print('complete.')
prior_data.drop('order_id_',axis = 1, inplace=True)
#prior_data.head(4)

In [ ]:
print('add order info to train_data')
train_data = train_data.join(orders,on='order_id',rsuffix='_')
train_data.drop('order_id_',axis = 1, inplace=True)
print('complete.')

In [ ]:
users = pd.DataFrame()
users['all_products'] = prior_data.groupby(['user_id'])['product_id'].apply(set)
users['user_id'] = users.index

In [ ]:
print(users.shape)
print(users.head())

d=dict()
e=dict()
z=0
y=0
train_users = pd.Series(train_data.groupby('user_id').user_id.apply(set))

for row in users.itertuples():
    if row.user_id in train_users:
        for prod in row.all_products:
            #generate train with labels data set
            if z%1000000==1: print('z: ' + str(z))
            up_key = row.user_id*100000 + prod
            label=0
            #if (row.user_id,prod) in labels.index: 
            #    label = labels.loc[row.user_id,prod]['reordered']
            d[z]=[row.user_id,prod]
            z+=1
    else:
          for prod in row.all_products:
            #generate test data set
            if y%1000000==1: print('y: ' + str(y)) 

            e[y]=[row.user_id,prod]
            y+=1      

#del train_users
print('complete')
%time

In [ ]:
print('converting train')
df_train = pd.DataFrame.from_dict(d,orient='index')
df_train.columns=['user_id','product_id']
#del d

print('converting test')
df_test = pd.DataFrame.from_dict(e,orient='index')
df_test.columns=['user_id','product_id']
#del e

df_test.head()
#df.head()

In [ ]:
print(df_train.nunique())
df_test.nunique()

In [ ]:
print('starting')
#generate labels
labels = train_data[train_data.reordered == 1][['user_id','order_id','product_id','reordered']]


labels['user_prod_key'] = labels.product_id +    labels.user_id*100000
df_train['user_prod_key']=df_train.product_id + df_train.user_id*100000

labels.set_index('user_prod_key',inplace=True)
df_train['label']= df_train.user_prod_key.map(labels.reordered)
print(labels.head())
df_train.head()

In [ ]:
#print(df_train.reordered.sum())
#labels.reordered.sum()
df_train['label'] = df_train.label.apply(lambda x: 0 if pd.isnull(x) else x)
df_train.head()

In [ ]:
#map the train and test orders to user_id
orders_map_test = orders[orders.eval_set == 'test']
orders_map_train = orders[orders.eval_set == 'train']
orders_map_test.set_index('user_id',inplace=True)
orders_map_train.set_index('user_id',inplace=True)
#print(orders_map.groupby('eval_set').order_id.nunique())
#print(orders_map.dtypes)

In [ ]:
print('add feature columns to train data')
print('order features')

df_train['order_id'] = df_train.user_id.map(orders_map_train.order_id).astype('int32')

df_train['order_number'] = df_train.order_id.map(orders.order_number)
df_train['order_dow'] = df_train.order_id.map(orders.order_dow)
df_train['order_hour_of_day'] = df_train.order_id.map(orders.order_hour_of_day)
df_train['days_since_prior_order'] = df_train.order_id.map(orders.days_since_prior_order)

df_train.sample(n=5).head(5)

In [ ]:
print('add feature columns to test data')
print('order features')

#df_test.to_csv('df_test.csv')
#orders_map.to_csv('orders_map.csv')


df_test['order_id'] = df_test.user_id.map(orders_map_test.order_id).astype('int32')

df_test['order_number'] = df_test.order_id.map(orders.order_number)
df_test['order_dow'] = df_test.order_id.map(orders.order_dow)
df_test['order_hour_of_day'] = df_test.order_id.map(orders.order_hour_of_day)
df_test['days_since_prior_order'] = df_test.order_id.map(orders.days_since_prior_order)

df_test.sample(n=5).head(5)

In [ ]:
%matplotlib inline
import lightgbm as lgb
#del prior_data
#del users

f_to_use=['order_number','order_dow','order_hour_of_day','days_since_prior_order']

print('formating for lgb')
d_train = lgb.Dataset(df_train[f_to_use],
                      label=df_train.label,
                      categorical_feature=['aisle_id', 'department_id'])  # , 'order_hour_of_day', 'dow'

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 100

print('light GBM train')
bst = lgb.train(params, d_train, ROUNDS)
lgb.plot_importance(bst, figsize=(9,20))
del d_train

In [ ]:
aa

In [19]:
print('light GBM predict')
preds = bst.predict(df_test[f_to_use])
print('Predictions complete')

In [20]:
import pickle
# now you can save it to a file
with open('pickled_model.pkl', 'wb') as f:
    pickle.dump(bst, f)

# and later you can load it
#with open('filename.pkl', 'rb') as f:
#    clf = pickle.load(f)

In [21]:
z=0
threshold = .1
submission = dict()

df_test['pred'] = preds

for row in df_test.itertuples():
    if z%1000000==1: print(z)
    if row.order_id in submission: 
        if preds[z] > threshold:
            try: submission[row.order_id] += ' ' + str(row.product_id)
            except: submission[row.order_id] = str(row.product_id)
    else: 
        submission[row.order_id] = ''
    z+=1

#convert missing products to none
for row in submission:
    if submission[row]=='': submission[row] = 'None'
#submission
print ('completo')

In [22]:
print('converting submission')
submission2 = pd.DataFrame.from_dict(submission,orient='index')
submission2.reset_index(inplace=True)
submission2.columns=['order_id','products']
print(submission2.shape)
submission2.to_csv('submission.csv')
#del submission
submission2